# Load Google Air Quality Token

Obtaining api token: https://developers.google.com/maps/documentation/air-quality/get-api-key?hl=es-419

Please execute the following command to install the required packages:

```bash
pip install -r requirements.txt 
```

before running the notebook

In [52]:
from dotenv import load_dotenv

load_dotenv()

True

# Air Quality Class for data from Google Air Quality API

In [53]:
import httpx
import os
from enum import Enum

class ExtraComputations(str, Enum):
    EXTRA_COMPUTATION_UNSPECIFIED = "EXTRA_COMPUTATION_UNSPECIFIED"
    LOCAL_AQI = "LOCAL_AQI"
    HEALTH_RECOMMENDATIONS = "HEALTH_RECOMMENDATIONS"
    POLLUTANT_ADDITIONAL_INFO = "POLLUTANT_ADDITIONAL_INFO"
    DOMINANT_POLLUTANT_CONCENTRATION = "DOMINANT_POLLUTANT_CONCENTRATION"
    POLLUTANT_CONCENTRATION = "POLLUTANT_CONCENTRATION"


class AirQuality:
    base_url: str
    headers: dict
    query_params: dict
    http_client: httpx.AsyncClient

    def __init__(self, base_url: str = "https://airquality.googleapis.com/v1") -> None:
        self.base_url = base_url
        self.headers = {"Content-Type": "application/json"}
        self.query_params = {"key": os.getenv("GOOGLE_AIR_QUALITY_TOKEN")}
        self.http_client = httpx.AsyncClient()
    
    async def get_current_conditions(self, latitude: float, longitude: float, extra_computations: list[ExtraComputations]) -> dict | None:
        url = f"{self.base_url}/currentConditions:lookup"
        
        payload = {
            "location": {
                "latitude": latitude,
                "longitude": longitude
            },
            "extraComputations": extra_computations
        }

        try:
            response = await self.http_client.post(
                url=url,
                headers=self.headers,
                params=self.query_params,
                json=payload
            )
            response.raise_for_status()
            return response.json()
        except Exception as e:
            print(f"Current conditions error for lat={latitude},long={longitude}. Error={e}")
            return None

    async def get_history_conditions(
        self,
        latitude: float,
        longitude: float,
        hours: int,
        extra_computations: list[ExtraComputations]
    ) -> list[dict]:
        
        url = f"{self.base_url}/history:lookup"

        payload = {
            "hours": hours,
            "pageSize": 72, # default value: 72
            "pageToken": "",
            "location": {
                "latitude": latitude,
                "longitude": longitude
            },
            "extraComputations": extra_computations
        }

        points = []
        next_page_token = True
        # Start request
        while next_page_token:
            response = await self.http_client.post(
                url=url,
                headers=self.headers,
                params=self.query_params,
                json=payload
            )

            data: dict = response.json()
            points.extend(data["hoursInfo"])
            next_page_token = data.get("nextPageToken") 
            payload.update({"pageToken": next_page_token})

        return points


# Load districts with points based on Metropolitan Arequipa

In [54]:
import json

with open("arequipa-districst-points.json", "r") as f:
    districts: dict = json.loads(f.read())


# Get history data and dumps per district and point into a JSONL

In [55]:
air_quality = AirQuality()

In [56]:
import asyncio
import pathlib

air_quality_dir = pathlib.Path("air-data")

for district_name, points in districts.items():
    print(f"Getting air quality data in {district_name} for {len(points)} points ...")
    tasks = []
    for point in points:
        tasks.append(air_quality.get_history_conditions(
            latitude=point[0],
            longitude=point[1],
            hours=720,
            extra_computations=[ExtraComputations.POLLUTANT_CONCENTRATION, ExtraComputations.HEALTH_RECOMMENDATIONS],
        ))

    results = await asyncio.gather(*tasks, return_exceptions=True)
    for (i, result), point in zip(enumerate(results), points):
        if isinstance(result, Exception):
            print(f"{district_name} couldn't get air quality data. Skip it, error {result}")
            continue

        district_name_normalized = district_name.strip().replace(" ", "").replace(",", "_").lower()
        fname = air_quality_dir / pathlib.Path(f"{district_name_normalized}_{i}.jsonl")

        with open(fname, "w") as f:
            for record in result:
                record["latitude"] = point[0]
                record["longitude"] = point[1]
                record["point"] = f"{district_name_normalized}_{i}"
                f.write(json.dumps(record) + "\n")


Getting air quality data in Arequipa, Arequipa, Peru for 3 points ...
Getting air quality data in Alto Selva Alegre, Arequipa, Peru for 5 points ...
Getting air quality data in Cayma, Arequipa, Peru for 5 points ...
Getting air quality data in Cerro Colorado, Arequipa, Peru for 5 points ...
Getting air quality data in Characato, Arequipa, Peru for 5 points ...
Getting air quality data in Chiguata, Arequipa, Peru for 5 points ...
Getting air quality data in Jacobo Hunter, Arequipa, Peru for 4 points ...
Getting air quality data in José Luis Bustamante y Rivero, Arequipa, Peru for 2 points ...
Getting air quality data in La Joya, Arequipa, Peru for 5 points ...
Getting air quality data in Mariano Melgar, Arequipa, Peru for 4 points ...
Getting air quality data in Miraflores, Arequipa, Peru for 2 points ...
Getting air quality data in Mollebaya, Arequipa, Peru for 5 points ...
Getting air quality data in Paucarpata, Arequipa, Peru for 5 points ...
Getting air quality data in Pocsi, Arequi